In [1]:
#!/usr/bin/env python3
import sys
import os
# Add the line_solver package to path if running as script
if '__file__' in globals():
    sys.path.insert(0, os.path.join(os.path.dirname(__file__), '..', '..', '..'))

from line_solver import *
import numpy as np

In [2]:
# CDF Response Time Analysis Example 3
# Open network with two classes, comparing Fluid and JMT transient CDF

model = Network('myModel')

# Block 1: nodes
node = [None] * 4  # using 0-based indexing
node[0] = Source(model, 'Source')
node[1] = Queue(model, 'Queue1', SchedStrategy.FCFS)
node[2] = Queue(model, 'Queue2', SchedStrategy.FCFS)
node[3] = Sink(model, 'Sink')

# Block 2: classes
jobclass = [None] * 2  # using 0-based indexing
jobclass[0] = OpenClass(model, 'Class1', 0)
jobclass[1] = OpenClass(model, 'Class2', 0)

# Arrival processes
node[0].set_arrival(jobclass[0], Exp.fit_mean(4.0))  # Source,Class1
node[0].set_arrival(jobclass[1], Exp.fit_mean(4.0))  # Source,Class2

# Service processes
node[1].set_service(jobclass[0], Exp.fit_mean(1.0))  # Queue1,Class1
node[1].set_service(jobclass[1], Exp.fit_mean(1.0))  # Queue1,Class2

node[2].set_service(jobclass[0], Exp.fit_mean(1.0))  # Queue2,Class1
node[2].set_service(jobclass[1], Exp.fit_mean(1.0))  # Queue2,Class2

# Block 3: topology - matching MATLAB implementation
P = model.init_routing_matrix()

# MATLAB: P{1,1}(1,2) = 1 - Class1->Class1: Source->Queue1
P.set(jobclass[0], jobclass[0], node[0], node[1], 1.0)

# MATLAB: P{1,2}(2,3) = 1 - Class1->Class2: Queue1->Queue2
P.set(jobclass[0], jobclass[1], node[1], node[2], 1.0)

# MATLAB: P{2,1}(3,4) = 1 - Class2->Class1: Queue2->Sink
P.set(jobclass[1], jobclass[0], node[2], node[3], 1.0)

# MATLAB: P{2,2}(1,2) = 1 - Class2->Class2: Source->Queue1
P.set(jobclass[1], jobclass[1], node[0], node[1], 1.0)

# MATLAB: P{2,1}(2,3) = 1 - Class2->Class1: Queue1->Queue2
P.set(jobclass[1], jobclass[0], node[1], node[2], 1.0)

# MATLAB: P{1,2}(3,4) = 1 - Class1->Class2: Queue2->Sink
P.set(jobclass[0], jobclass[1], node[2], node[3], 1.0)

model.link(P)

In [3]:
# Solve with JMT solver - aligned with MATLAB test scenario  
jmtoptions = JMT.default_options()
jmtoptions.samples = int(10000)  # Aligned with MATLAB: samples=1e4
jmtoptions.seed = 23000  # Aligned with MATLAB: seed=23000
RDsim = JMT(model, jmtoptions).get_tran_cdf_respt()  # Using getTranCdfRespT like MATLAB

# Solve with Fluid solver - aligned with MATLAB test scenario
options = FLD.default_options()  
options.iter_max = 300  # Aligned with MATLAB: iter_max=300
RDfluid = FLD(model, options).cdf_respt()

JMT Model: /tmp/workspace/jsim/2773839395477449200/jmodel.jsim
JMT Model: /tmp/workspace/jsim/12888819071825388323/jmodel.jsim
INFO: Added 20 refined points between t=0.000000 and t=0.004002
INFO: Added 20 refined points between t=0.000000 and t=0.004002
INFO: Added 20 refined points between t=0.000000 and t=0.004002
INFO: Added 20 refined points between t=0.000000 and t=0.004002


In [ ]:
# Plotting and comparison - matching MATLAB layout (4 plots in 2x2 grid)
import matplotlib.pyplot as plt

# Create 2x2 subplot grid for the 4 plots (2 stations x 2 classes)
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

print(f"Number of stations: {model.get_number_of_stations()}")
print(f"Station range: {list(range(1, model.get_number_of_stations() - 1))}")

plot_idx = 0
# Iterate over both queue stations 1 and 2 
for i in range(1, model.get_number_of_stations() - 1):  # Should be [1, 2] for stations Queue1, Queue2
    for c in range(model.get_number_of_classes()):  # classes [0, 1]
        row = plot_idx // 2
        col = plot_idx % 2
        
        print(f"Processing station {i}, class {c} -> plot position [{row}, {col}]")
        
        if i < len(RDsim) and c < len(RDsim[i]) and RDsim[i][c] is not None and len(RDsim[i]) > c and RDfluid[i][c] is not None:
            axes[row, col].loglog(RDsim[i][c][:, 1], 1 - RDsim[i][c][:, 0], 'r--', label='sim')
            axes[row, col].loglog(RDfluid[i][c][:, 1], 1 - RDfluid[i][c][:, 0], 'b-.', label='fluid')
            axes[row, col].legend(loc='lower left')
            axes[row, col].set_title(f'RespT Tail: Node {i}, Class {c+1}')
            print(f"  -> Plotted data for station {i}, class {c}")
        else:
            axes[row, col].set_title(f'RespT Tail: Node {i}, Class {c+1} (no data)')
            print(f"  -> No data for station {i}, class {c}")
        
        plot_idx += 1

plt.tight_layout()
plt.show()

# Calculate average response times from CDFs - matching MATLAB indexing
AvgRespTfromCDFfluid = np.zeros((model.get_number_of_stations(), model.get_number_of_classes()))
AvgRespTfromCDFsim = np.zeros((model.get_number_of_stations(), model.get_number_of_classes()))

for i in range(1, model.get_number_of_stations()):  # stations 1,2,3 like MATLAB i=2:end
    for c in range(model.get_number_of_classes()):
        # From Fluid CDF
        if i < len(RDfluid) and c < len(RDfluid[i]) and RDfluid[i][c] is not None and len(RDfluid[i][c]) > 1:
            diffs = np.diff(RDfluid[i][c][:, 0])
            values = RDfluid[i][c][1:, 1]
            AvgRespTfromCDFfluid[i, c] = np.sum(diffs * values)
        
        # From Simulation CDF
        if i < len(RDsim) and c < len(RDsim[i]) and RDsim[i][c] is not None and len(RDsim[i][c]) > 1:
            diffs = np.diff(RDsim[i][c][:, 0])
            values = RDsim[i][c][1:, 1]
            AvgRespTfromCDFsim[i, c] = np.sum(diffs * values)

print('AvgRespTfromCDFfluid =')
print(AvgRespTfromCDFfluid)
print('AvgRespTfromCDFsim =')
print(AvgRespTfromCDFsim)